In [1]:
import time
import numpy as np
from snnpy import *
from sklearn.neighbors import BallTree
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree
from bf_search import *

In [2]:
def bvecs_read(fname):
    a = np.fromfile(fname, dtype=np.int32, count=1)
    b = np.fromfile(fname, dtype=np.uint8)
    d = a[0]
    return b.reshape(-1, d + 4)[:, 4:].copy()


def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy()


def fvecs_read2(fname):
    return ivecs_read(fname).view('float32')

#### sift

In [3]:
sift_train = fvecs_read2('Euclidean_data/sift/sift_learn.fvecs')
sift_query = fvecs_read2('Euclidean_data/sift/sift_query.fvecs')


sift_kdtree_index_timing = list()
sift_balltree_index_timing = list()
sift_snn_index_timing = list()

sift_bf_run_timing1 = list()
sift_bf_run_timing2 = list()
sift_kdtree_run_timing = list()
sift_balltree_run_timing = list()
sift_snn_run_timing = list()


In [4]:
radius = [210, 230, 250, 270, 290] # test various radius

In [5]:
# Brute force 1
for R in radius:
    st = time.time()
    neigh = NearestNeighbors(radius=R, algorithm='brute')
    neigh.fit(sift_train)
    for j in range(sift_query.shape[0]): 
        ind = neigh.radius_neighbors(
           sift_query[j:j+1], radius=R, return_distance=True
        )
    et = time.time() - st
    sift_bf_run_timing1.append(et)
    print("brute force 1 query time:", et)
print()
    

# Brute force 2
for R in radius:
    st = time.time()
    for j in range(sift_query.shape[0]): 
        ind = bf_radius_fairness(sift_query[j], sift_train, R, return_distance=False)
    et = time.time() - st
    sift_bf_run_timing2.append(et)
    print("brute force 2 query time:", et)
print()
    
    
# KDtree
st = time.time()
tree = KDTree(sift_train)   
et = time.time() - st
sift_kdtree_index_timing.append(et)
print("kdtree build time:", et)
for R in radius:
    st = time.time()
    for j in range(sift_query.shape[0]):
        ind = tree.query_radius(sift_query[j:j+1], r=R, return_distance=False)
    et = time.time() - st
    sift_kdtree_run_timing.append(et)
    print("kdtree query time:", et)
print()

# Balltree
st = time.time()
tree = BallTree(sift_train)   
et = time.time() - st
sift_balltree_index_timing.append(et)
print("balltree build time:", et)

for R in radius:
    st = time.time()
    for j in range(sift_query.shape[0]):
        ind = tree.query_radius(sift_query[j:j+1], r=R, return_distance=False)
    et = time.time() - st
    sift_balltree_run_timing.append(et)
    print("balltree query time:", et)

print()


# SNN
st = time.time()
snn = build_snn_model(sift_train)
et = time.time() - st
sift_snn_index_timing.append(et)
print("snn build time:", et)


for R in radius:
    st = time.time()
    for j in range(sift_query.shape[0]):
        ind = snn.query_radius(sift_query[j], R)
    et = time.time() - st
    sift_snn_run_timing.append(et)
    print("snn query time:", et)

brute force 1 query time: 758.1585776805878
brute force 1 query time: 782.4455168247223
brute force 1 query time: 860.3478307723999
brute force 1 query time: 801.2675459384918
brute force 1 query time: 697.6933727264404

brute force 2 query time: 161.0031614303589
brute force 2 query time: 161.463858127594
brute force 2 query time: 162.8732373714447
brute force 2 query time: 161.74675011634827
brute force 2 query time: 161.65616154670715

kdtree build time: 3.2916934490203857
kdtree query time: 457.054354429245
kdtree query time: 468.6130566596985
kdtree query time: 502.95971846580505
kdtree query time: 541.311770439148
kdtree query time: 587.9517521858215

balltree build time: 2.9213178157806396
balltree query time: 351.1425702571869
balltree query time: 383.679315328598
balltree query time: 407.50739073753357
balltree query time: 427.6475508213043
balltree query time: 445.45238876342773

snn build time: 0.17904901504516602
snn query time: 45.25027346611023
snn query time: 45.57203340

In [6]:
with open('result/sift_kdtree_index_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_kdtree_index_timing))

with open('result/sift_balltree_index_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_balltree_index_timing))

with open('result/sift_snn_index_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_snn_index_timing))
    
with open('result/sift_bf_run_timing1_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_bf_run_timing1))

with open('result/sift_bf_run_timing2_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_bf_run_timing2))
    
with open('result/sift_kdtree_run_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_kdtree_run_timing))

with open('result/sift_balltree_run_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_balltree_run_timing))

with open('result/sift_snn_run_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_snn_run_timing))
    